In [1]:
import math

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

/usr/local/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
# Importing and Cleaning:

data = pd.read_excel("new_york_2013.xls", index_col = "Table 8", skiprows = [1, 2, 3, 4], skipfooter = 3)

data = data.rename(columns = {'Unnamed: 1': 'Population', 'Unnamed: 2': 'Violent_Crime', 'Unnamed: 3': 'Murder',
                       'Unnamed: 4': 'Rape_1', 'Unnamed: 5': 'Rape_2', 'Unnamed: 6': 'Robbery',
                       'Unnamed: 7': 'Agg_Assault', 'Unnamed: 8': 'Prop_Crime', 'Unnamed: 9': 'Burglary',
                       'Unnamed: 10': 'Larceny_Theft', 'Unnamed: 11': 'Vehicle_Theft', 'Unnamed: 12': 'Arson'})

In [3]:
# Feature Engineering: (Also had to manipulate data_2 a bit because of a 'copy error')

data_2 = pd.DataFrame(data[['Population', 'Murder', 'Robbery', 'Prop_Crime']])

data_2['Cat_Murder'] = data_2.Murder > 0
data_2['Cat_Robbery'] = data_2.Robbery > 0
data_2['Population*2'] = data_2.Population**2
data_2['Mrdr_per_Capita'] = (data_2.Murder / data_2.Population)*5000
data_2['Rob_per_Capita'] = (data_2.Robbery / data_2.Population)*5000
data_2['PropCrime_per_Capita'] = (data_2.Prop_Crime / data_2.Population)*5000


### First Model

In [4]:
no_robberies = pd.DataFrame(data_2[data_2['Robbery'] == 0])
robberies_sm = pd.DataFrame(data_2[(data_2['Robbery'] >= 1) & (data_2['Population'] <= 100000)])
robberies_lg = pd.DataFrame(data_2[(data_2['Robbery'] >= 1) & (data_2['Population'] >= 100000)])

In [5]:
model = pd.concat([no_robberies, robberies_sm, robberies_lg])

In [6]:
data_2.describe()

,Population,Murder,Robbery,Prop_Crime,Population*2,Mrdr_per_Capita,Rob_per_Capita,PropCrime_per_Capita
count,3.480000e+02,348.000000,348.000000,348.000000,3.480000e+02,348.000000,348.000000,348.000000
mean,4.003763e+04,1.566092,72.902299,792.606322,2.035547e+11,0.055101,1.883793,97.404954
std,4.500374e+05,18.303673,1031.032873,7659.724746,3.778876e+12,0.174159,3.579952,72.995665
min,5.260000e+02,0.000000,0.000000,0.000000,2.766760e+05,0.000000,0.000000,0.000000
25%,3.003000e+03,0.000000,0.000000,40.500000,9.018117e+06,0.000000,0.000000,45.521000
50%,7.233500e+03,0.000000,1.000000,112.500000,5.232568e+07,0.000000,0.736432,80.051048
75%,1.842750e+04,0.000000,5.000000,341.000000,3.397536e+08,0.000000,2.229189,128.603221
max,8.396126e+06,335.000000,19170.000000,141971.000000,7.049493e+13,1.137176,26.250394,618.120237


In [7]:
# Classifying the Data:

for x in model:
    model['Actual'] = (((model['Robbery'] == 0) & (model['PropCrime_per_Capita'] <= 79)) |
                       ((model['Robbery'] <= 11) & (model['Robbery'] >= 1)  & (model['PropCrime_per_Capita'] >= 164)) |
                       ((model['Robbery'] >= 12) & (model['PropCrime_per_Capita'] <= 241)))

In [8]:
# First Model:

for x in model:
    model['Test_1'] = (((model['Robbery'] == 0) & (model['PropCrime_per_Capita'] <= 45)) |
                       ((model['Robbery'] <= 4) & (model['Robbery'] >= 1)  & (model['PropCrime_per_Capita'] >= 151)) |
                       ((model['Robbery'] <= 5) & (model['Robbery'] >= 11)  & (model['PropCrime_per_Capita'] >= 300)) |
                       ((model['Robbery'] >= 11) & (model['PropCrime_per_Capita'] <= 150)) |
                       ((model['Robbery'] >= 600) & (model['PropCrime_per_Capita'] >= 150)))

#### Validating the Model

In [9]:
x = data_2.Population/100
lin_form = 'Prop_Crime ~ Robbery*x'

lm = smf.ols(formula = lin_form, data = data_2).fit()

In [10]:
print("Parameters: {}".format(lm.params))
print("P-Values: {}".format(lm.pvalues))
print("R-squared: {}".format(lm.rsquared))

Parameters: Intercept    33.632744
Robbery       6.978783
x             1.435682
Robbery:x    -0.000070
dtype: float64
P-Values: Intercept    9.615761e-02
Robbery      2.018976e-76
x            3.449308e-36
Robbery:x    4.017653e-65
dtype: float64
R-squared: 0.9985148599885865


### Using it on another test set

In [11]:
cali_data = pd.read_excel("california_2013.xls", index_col = "Table 8", skiprows = [1, 2, 3, 4], skipfooter = 3)

In [12]:
cali_data = cali_data.rename(columns = {'Unnamed: 1': 'Population', 'Unnamed: 2': 'Violent_Crime', 'Unnamed: 3': 'Murder',
                       'Unnamed: 4': 'Rape_1', 'Unnamed: 5': 'Rape_2', 'Unnamed: 6': 'Robbery',
                       'Unnamed: 7': 'Agg_Assault', 'Unnamed: 8': 'Prop_Crime', 'Unnamed: 9': 'Burglary',
                       'Unnamed: 10': 'Larceny_Theft', 'Unnamed: 11': 'Vehicle_Theft', 'Unnamed: 12': 'Arson'})

cali_data = cali_data[['Population', 'Murder', 'Robbery', 'Prop_Crime']]

In [13]:
y = cali_data.Population/100
lin_form = 'Prop_Crime ~ Robbery*y'

lm2 = smf.ols(formula = lin_form, data = cali_data).fit()

In [14]:
print("Parameters: {}".format(lm2.params))
print("P-Values: {}".format(lm2.pvalues))
print("R-squared: {}".format(lm2.rsquared))

Parameters: Intercept   -65.392578
Robbery       5.831921
y             2.151199
Robbery:y    -0.000140
dtype: float64
P-Values: Intercept     2.246368e-01
Robbery      2.424456e-117
y            1.644232e-136
Robbery:y     4.484968e-58
dtype: float64
R-squared: 0.9683705689326801


### Trying to tighten the model up a bit:

In [19]:
z = cali_data.Population/500
lin_form3 = 'Prop_Crime ~ Robbery*z'
lm3 = smf.ols(formula = lin_form3, data = cali_data).fit()

In [20]:
print("Parameters: {}".format(lm3.params))
print("P-Values: {}".format(lm3.pvalues))
print("R-squared: {}".format(lm3.rsquared))

Parameters: Intercept   -65.392578
Robbery       5.831921
z            10.755994
Robbery:z    -0.000702
dtype: float64
P-Values: Intercept     2.246368e-01
Robbery      2.424456e-117
z            1.644232e-136
Robbery:z     4.484968e-58
dtype: float64
R-squared: 0.9683705689326803


Here, I tried to change the number dividing the population (in the hopes of predicting Poperty Crime as a ratio between Robbery and Population, but the change in the resulting P-values are very very small.  Too small for us to notice unless you look hard at the decimals on the far right hand side.  As such, it looks like you would need to find a parameter that has some stronger correspondence.